In [34]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import requests
from bs4 import BeautifulSoup

from PIL import Image
import io

In [35]:
browser = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST chromedriver version for 91.0.4472 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/91.0.4472.101/chromedriver_linux64.zip
Driver has been saved in cache [/home/murakami/.wdm/drivers/chromedriver/linux64/91.0.4472.101]


In [37]:
url = 'https://suumo.jp/jj/bukken/ichiran/JJ010FJ001/?ar=020&bs=021&ta=06&jspIdFlg=patternShikugun&kb=1&kt=9999999&tb=0&tt=9999999&hb=0&ht=9999999&ekTjCd=&ekTjNm=&tj=0&cnb=0&cn=9999999'
browser.get(url)

count = 0

In [39]:
house_data_list = []

In [112]:
url = browser.current_url
res = requests.get(url)
soup = BeautifulSoup(res.content, 'html.parser')
a_elems = soup.find_all('div', attrs={'class' : 'property_unit-header'})

**各ページのURLを取得する関数**
1. SUUMOの物件情報(indexページ)は、基本的に内部リンク（'https:~がない"）形式でHTMLが作成されているため、内部リンクのみ全体のページから取得する
2. 取得したURLをリスト形式にまとめる
3. URLのリストを返す関数を作成する

In [113]:
def get_urls(soup) -> urls:list:
    """
    一覧のページからURL個別ページのURLを取得する関数
    """
    h2_elems = soup.find_all('h2', attrs={'class' : 'property_unit-title'})
    
    urls = []
    
    for h2_elem in h2_elems:
        a_elem = h2_elem.find('a')
        url = a_elem.attrs['href']
        
        urls.append(url)
        
    return urls

**各ページの物件情報を収集する関数**
- 入力は各ページの内部リンクのURL（シングル）
- 出力は各ページのsoup

In [264]:
def get_page_soup(internal_url:str):
    """
    各物件へのリンクを作成して、アクセスして、各ページのsoupを出力する
    """
    # ページ内リンクをドメインと結びつけて直にアクセスできるリンクに変換している
    page_url = 'https://suumo.jp' + internal_url
    page_res = requests.get(page_url)
    page_soup = BeautifulSoup(page_res.content, 'html.parser')
    
    return page_soup

**各ページの物件情報を収集する関数**
- 入力は各ページのsoup
- 出力は各ページの物件情報が格納された辞書

In [265]:
def get_house_details(page_soup) -> dict:
    """[summary]　
    　その後に各物件のページから「物件情報」にアクセスして、「物件情報」のテーブルまるごと取得する。
    Args
        internal_url[str]:SUUMOの各物件の内部リンク
    """
    #物件詳細のページへのリンクの取得
    house_details_a_elem = page_soup.find('a', attrs={'class' : 'tabOutline2'})
    
    if house_details_a_elem is None:
        house_details_a_elem = page_soup.find('a', attrs={'class' : 'tabOutline'})
        print(house_details_a_elem)
        
    house_details_url = house_details_a_elem.attrs['href']
    
    #物件詳細のページへのアクセス    
    house_details_res = requests.get(house_details_url)
    house_details_soup = BeautifulSoup(house_details_res.content, 'html.parser')
    
    # テーブルの取得
    house_details_info = house_details_soup.find('table', {'class': 'pCell10'})

    return house_details_info

**各ページの物件情報を収集する関数**
- 入力はtable要素
- 出力はth要素がKey、td要素がValueになった辞書

In [374]:
def extract_table_data(table) -> dict:
    """
    　tableのHTMLからthとtd内の情報を取得する　
    """
    house_details_dict = {}

    for row in table.find_all('tr'):
        data_head = row.find_all('th')
        data_value = row.find_all('td')

        data_head_01 = data_head[0].text.replace('\n', '').replace('ヒント', '')
        data_value_01 = data_value[0].text.replace('\n', '').replace('\r', '').replace('\t', '')

        try:
            if data_head[1] is not None:
                data_head_02 = data_head[1].text.replace('\n', '').replace('ヒント', '')
                data_value_02 = data_value[1].text.replace('\n', '').replace('\r', '').replace('\t', '').replace('\u3000', '').replace('\xa0', '').replace('[乗り換え案内]', '')

                #print(data_head_01, data_value_01, data_head_02, data_value_02)

                house_details_dict[data_head_01] = data_value_01
                house_details_dict[data_head_02] = data_value_02
        except IndexError:
            house_details_dict[data_head_01] = data_value_01
            
    return house_details_dict

**各物件のタイトルとコメントを取得する関数**

In [349]:
def get_title_and_comment(page_soup):
    """
    サイト上部にあるタイトルとコメントを取得する関数
    """
    try:
        house_title = page_soup.find('h2', attrs = {'class' : 'fs16'})
        house_comment = page_soup.find('p', attrs = {'class' : 'fs14'})
        house_dict = {'title':house_title.text, 'comment':house_comment.text}
    except:
        house_dict = {'title':'', 'comment':''}
    return house_dict

In [342]:
page_soup = get_page_soup(urls[1]) 
table = get_house_details(page_soup)
house_info_dict = extract_table_data(table)
house_text_dict = get_title_and_comment(page_soup)


# house_comment = page_soup.find('p', attrs = {'class' : 'fs14'})
# house_title = page_soup.find('h2', attrs = {'class' : 'fs16'})
# print(house_comment.text, '\n', house_title.text)

販売スケジュール - イベント情報 -
所在地 山形県東村山郡中山町大字長崎 交通 ＪＲ左沢線「羽前長崎」歩22分
販売戸数 1戸 総戸数 -
価格 1098万円[ □支払シミュレーション ] 最多価格帯 -
私道負担・道路 無、北6ｍ幅、西6ｍ幅 諸費用 -
間取り 4LDK 建物面積 84.84m2（25.66坪）（登記）
土地面積 164.02m2（49.61坪）（登記） 建ぺい率・容積率 60％・200％
完成時期(築年月) 1985年6月 入居時期 相談
土地の権利形態 所有権 構造・工法 木造2階建
施工 - リフォーム 2020年2月完了水回り設備交換：キッチン・浴室・トイレ※年月は一番古いリフォーム箇所を表します
用途地域 １種住居 地目 -
{'title': 'リニューアル工事しました!!', 'comment': '・駐車場拡張・ユニットバス交換・換気扇取替・トイレ交換・CF張替え・かぎ交換・白あり点検・キッチン交換・洗面化粧台交換・クロス張替え・床一部張替え・建具調整・照明器具交換・畳表替え・障子、ふすま張替え・ハウスクリーニング…他'}


**家の画像の取得**


In [375]:
def get_house_img(page_soup, house_id):
    """
    各ページの写真を取得して、写真をHouseId_IMGIDの形式で保存する。
    """
    imgs = page_soup.find_all('img')
    img_list = list()
    img_id = 0
    # 以下で始まるのはIMGタグだがアクセスできないため排除する
    un_img_signal = 'img01.suumo.com'
    
    for img in imgs:
        try:
            img_url = img['rel']
            img_tag = img['alt']
        except KeyError:
            img_url = un_img_signal
            img_tag = ""

        if not img_url in un_img_signal:
            img_name = str(house_id) + '_' + str(img_id)
            img_id += 1
            img = Image.open(io.BytesIO(requests.get(img_url).content))
            img.save(f'imgs/{img_name}.jpg')


            img_list.append({'house_id':house_id, 'img_id':img_id, 'img_tag':img_tag, 'img_name':img_name})
    
    return img_list

**Index1ページ分のURL**

- ここのループでは、Index1ページ分のURLをすべて取ってきている

In [376]:
house_info = []
house_id = 0

for url in urls:
    #print(url)
    page_soup = get_page_soup(url)
    table = get_house_details(page_soup)
    house_info_dict = extract_table_data(table)
    house_text_dict = get_title_and_comment(page_soup)
    house_img_list = get_house_img(page_soup, house_id)
    
    house_dict = {'House_ID': house_id, 'text':house_text_dict, 'info':house_info_dict, 'imgs':house_img_list}
    house_info.append(house_dict)
    


<a class="bt btOl tabOutline" href="https://suumo.jp/chukoikkodate/yamagata/sc_yonezawa/nc_96375651/bukkengaiyo/?fmlg=t001">物件概要</a>
<a class="bt btOl tabOutline" href="https://suumo.jp/chukoikkodate/yamagata/sc_tsuruoka/nc_95054357/bukkengaiyo/?fmlg=t001">物件概要</a>
<a class="bt btOl tabOutline" href="https://suumo.jp/chukoikkodate/yamagata/sc_yonezawa/nc_96852684/bukkengaiyo/?fmlg=t001">物件概要</a>
<a class="bt btOl tabOutline" href="https://suumo.jp/chukoikkodate/yamagata/sc_sakata/nc_95856457/bukkengaiyo/?fmlg=t001">物件概要</a>
<a class="bt btOl tabOutline" href="https://suumo.jp/chukoikkodate/yamagata/sc_yonezawa/nc_95750706/bukkengaiyo/?fmlg=t001">物件概要</a>
<a class="bt btOl tabOutline" href="https://suumo.jp/chukoikkodate/yamagata/sc_obanazawa/nc_96851112/bukkengaiyo/?fmlg=t001">物件概要</a>
<a class="bt btOl tabOutline" href="https://suumo.jp/chukoikkodate/yamagata/sc_tsuruoka/nc_95793061/bukkengaiyo/?fmlg=t001">物件概要</a>
<a class="bt btOl tabOutline" href="https://suumo.jp/chukoikkodate/yam

In [377]:
house_info

[{'House_ID': 0,
  'text': {'title': '令和２年２月末内装リフォーム済み！閑静な住宅地！国道112号線へのアクセス良し！',
   'comment': '【リフォーム内容】ユニットバス交換・換気扇取替・トイレ交換・CF貼替・キッチンシンク交換・洗面化粧台交換・クロス貼替・床一部張替・建具調整・畳表替・障子張替・襖張替・照明器具交換・ハウスクリーニング駐車２台可、即入居可、内装リフォーム、南向き、陽当り良好、角地、和室'},
  'info': {'販売スケジュール': '-',
   'イベント情報': '-',
   '所在地': '山形県東村山郡中山町大字長崎',
   '交通': 'ＪＲ左沢線「羽前長崎」歩22分',
   '販売戸数': '1戸',
   '総戸数': '1戸',
   '価格': '1098万円[ □支払シミュレーション ]',
   '最多価格帯': '-',
   '私道負担・道路': '無、西6ｍ幅（接道幅9ｍ）、北6ｍ幅（接道幅9ｍ）',
   '諸費用': '-',
   '間取り': '4LDK',
   '建物面積': '84.84m2（登記）',
   '土地面積': '164.02m2（登記）',
   '建ぺい率・容積率': '60％・200％',
   '完成時期(築年月)': '1985年6月',
   '入居時期': '即入居可',
   '土地の権利形態': '所有権',
   '構造・工法': '木造2階建',
   '施工': '-',
   'リフォーム': '2020年2月完了水回り設備交換：キッチン・浴室・トイレ※年月は一番古いリフォーム箇所を表します',
   '用途地域': '１種住居',
   '地目': '-',
   'その他制限事項': '-',
   'その他概要・特記事項': '設備：公営水道、個別浄化槽、個別LPG、駐車場：カースペース'},
  'imgs': [{'house_id': 0,
    'img_id': 1,
    'img_tag': '現地外観写真',
    'img_name': '0_0'},
   {'house_id': 0, 'img_id': 2, 'img_tag': '間取り図', 'img_name':